## DP-1 Oliver Adler demo

Nakoniec som sa rozhodol pre hľadanie implicitných vzťahov medzi vzormi. Používal som zatiaľ iba organizačné vzory.

In [1]:
import os
import pandas as pd
import numpy as np
from itertools import product, combinations
import transformers
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
from transformers import LongformerTokenizerFast, \
LongformerModel, LongformerTokenizer, LongformerConfig, Trainer, TrainingArguments, EvalPrediction, AutoTokenizer, AutoModel
from transformers.models.longformer.modeling_longformer import LongformerPreTrainedModel, LongformerClassificationHead

from tensorflow.keras import layers
from tensorflow.keras import losses

C:\Users\olive\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pre toto demo som použil vzory z jazyka ProjectManagementPatternLanguage a PiecemealGrowthPatternLanguage. Z každého vzoru vezmem jeho názov, počiatočný kontext (= problém ktorý rieši) a jeho konečný kontext. Text rozdeľujem podľa schémy, ktorú vzory z tejto kolekcie majú. 

In [123]:
directory = "patterns/ProjectManagmentPatternLanguage"
starting_contexts = []
resulting_contexts = []
pattern_names = []
pattern_languages = []
for filename in os.listdir(directory):
    print(filename)
    with open(directory+"/"+filename) as f:
        lines = f.readlines()
        pattern_names.append(lines[0].replace('\n',''))
        pattern_languages.append("ProjectManagmentPatternLanguage")
        starting_context = ""
        
        resulting_context = ""
        starting_context_flag = False
        resulting_context_flag = False
        for line in lines:
            #end of resulting context statement
            if "âœ¥" in line and resulting_context_flag:
                break
            
            #beginning of starting context statement
            if "âœ¥" in line and not starting_context_flag:
                starting_context_flag = True
                continue
                
            #beginning of resulting context statement
            if "Therefore:" in line and starting_context_flag:
                starting_context_flag = False
                resulting_context_flag = True
                continue

            if starting_context_flag and line != "\n":
                starting_context+=line

            if resulting_context_flag and line != "\n":
                resulting_context+=line
      
        starting_contexts.append(starting_context.translate({ord('\n'): None}))
        resulting_contexts.append(resulting_context.translate({ord('\n'): None}))
directory = "patterns/PiecemealGrowthPatternLanguage"
for filename in os.listdir(directory):
    with open(directory+"/"+filename) as f:
        lines = f.readlines()
        pattern_names.append(lines[0].replace('\n',''))
        pattern_languages.append("PiecemealGrowthPatternLanguage")
        starting_context = ""
        
        resulting_context = ""
        starting_context_flag = False
        resulting_context_flag = False
        for line in lines:
            #end of resulting context statement
            if "âœ¥" in line and resulting_context_flag:
                break
            
            #beginning of starting context statement
            if "âœ¥" in line and not starting_context_flag:
                starting_context_flag = True
                continue
                
            #beginning of resulting context statement
            if "Therefore:" in line and starting_context_flag:
                starting_context_flag = False
                resulting_context_flag = True
                continue

            if starting_context_flag and line != "\n":
                starting_context+=line

            if resulting_context_flag and line != "\n":
                resulting_context+=line
      
        starting_contexts.append(starting_context.translate({ord('\n'): None}))
        resulting_contexts.append(resulting_context.translate({ord('\n'): None}))

directory = "patterns/OrganizationalStyleLanguage"
for filename in os.listdir(directory):
    with open(directory+"/"+filename) as f:
        lines = f.readlines()
        pattern_names.append(lines[0].replace('\n',''))
        pattern_languages.append("OrganizationalStyleLanguage")
        starting_context = ""
        
        resulting_context = ""
        starting_context_flag = False
        resulting_context_flag = False
        for line in lines:
            #end of resulting context statement
            if "âœ¥" in line and resulting_context_flag:
                break
            
            #beginning of starting context statement
            if "âœ¥" in line and not starting_context_flag:
                starting_context_flag = True
                continue
                
            #beginning of resulting context statement
            if "Therefore:" in line and starting_context_flag:
                starting_context_flag = False
                resulting_context_flag = True
                continue

            if starting_context_flag and line != "\n":
                starting_context+=line

            if resulting_context_flag and line != "\n":
                resulting_context+=line
      
        starting_contexts.append(starting_context.translate({ord('\n'): None}))
        resulting_contexts.append(resulting_context.translate({ord('\n'): None}))
        
        
directory = "patterns/PeopleAndCodePatternLanguage"
for filename in os.listdir(directory):
    with open(directory+"/"+filename) as f:
        lines = f.readlines()
        pattern_names.append(lines[0].replace('\n',''))
        pattern_languages.append("PeopleAndCodePatternLanguage")
        starting_context = ""
        
        resulting_context = ""
        starting_context_flag = False
        resulting_context_flag = False
        for line in lines:
            #end of resulting context statement
            if "âœ¥" in line and resulting_context_flag:
                break
            
            #beginning of starting context statement
            if "âœ¥" in line and not starting_context_flag:
                starting_context_flag = True
                continue
                
            #beginning of resulting context statement
            if "Therefore:" in line and starting_context_flag:
                starting_context_flag = False
                resulting_context_flag = True
                continue

            if starting_context_flag and line != "\n":
                starting_context+=line

            if resulting_context_flag and line != "\n":
                resulting_context+=line
      
        starting_contexts.append(starting_context.translate({ord('\n'): None}))
        resulting_contexts.append(resulting_context.translate({ord('\n'): None}))

BuildPrototypes.txt
CommunityOfTrust.txt
CompletionHeadroom.txt
DayCare.txt
DeveloperControlsProcess.txt
DevelopmentEpisode.txt
DontInteruptAnInterrupt.txt
GetOnWithIt.txt
ImpliedRequirements.txt
IncrementalIntegration.txt
InformalLaborPlan.txt
InterruptsUnjamBlocking.txt
MercenaryAnalyst.txt
NamedStableBases.txt
PrivateWorld.txt
ProgrammingEpisode.txt
RecommitmentMeeting.txt
SacrificeOnePerson.txt
SizeTheSchedule.txt
SomeoneAlwaysMakesProgress.txt
TakeNoSmallSlips.txt
TeamPerTask.txt
WorkFlowsInward.txt
WorkQueue.txt
WorkSplit.txt


In [124]:
patterns=[]
for i,pattern_name in enumerate(pattern_names):
    patterns.append(pattern_name)
    patterns.append(starting_contexts[i])
    patterns.append(resulting_contexts[i])

In [125]:
d = {"pattern_names": pattern_names, 'pattern_language': pattern_languages,'starting_contexts': starting_contexts, "resulting_contexts": resulting_contexts}
df = pd.DataFrame(data=d)

In [126]:
df

,pattern_names,pattern_language,starting_contexts,resulting_contexts
0,Build Prototypes,ProjectManagmentPatternLanguage,A project must test requirements and design de...,Build an isolated prototype solution whose pur...
1,Community Of Trust,ProjectManagmentPatternLanguage,It is essential that the people in a team trus...,"Do things that explicitly demonstrate trust, s..."
2,Completion Headroom,ProjectManagmentPatternLanguage,Every project must commit to delivery on a few...,Project work group completion dates from remai...
3,Day Care,ProjectManagmentPatternLanguage,Your experts are spending all their time mento...,"Put one expert in charge of all the novices, l..."
4,Developer Controls Process,ProjectManagmentPatternLanguage,"A development culture, like any culture, can b...",Make the Developer the focal point of process ...
...,...,...,...,...
87,Smoke-Filled Room,PeopleAndCodePatternLanguage,An organization must make a timely decision ab...,Make the decision among power brokers as in th...
88,Standards Linking Locations,PeopleAndCodePatternLanguage,It is difficult for geographically distanced t...,Use standards to express architectural concern...
89,Stand-Up Meeting,PeopleAndCodePatternLanguage,"At times of fast change or high stress, it is ...",Hold short daily meetings with the entire team...
90,Subclass Per Team,PeopleAndCodePatternLanguage,Subsystem teams have differing interests and d...,"Where two subsystems collide in one class, fac..."


Zo vzorov vytváram dvojice, kde pre dvojicu vzorov X a Y, budem mať v dataframe ich názvy, počiatočný kontext vzoru X a konečný kontext vzoru Y 

In [127]:
model = SentenceTransformer('all-mpnet-base-v2')
df['starting_embedding'] = df['starting_contexts'].apply(model.encode)
df['resulting_embedding'] = df['resulting_contexts'].apply(model.encode)

In [128]:
pattern_pairs = list(combinations(df['pattern_names'].tolist(),2))
pattern_pairs

[('Build Prototypes', 'Community Of Trust'),
 ('Build Prototypes', 'Completion Headroom'),
 ('Build Prototypes', 'Day Care'),
 ('Build Prototypes', 'Developer Controls Process'),
 ('Build Prototypes', 'Development Episode'),
 ('Build Prototypes', "Don't Interrupt An Interrupt"),
 ('Build Prototypes', 'Get On With It'),
 ('Build Prototypes', 'Implied Requirements'),
 ('Build Prototypes', 'Incremental Integration'),
 ('Build Prototypes', 'Informal Labor Plan'),
 ('Build Prototypes', 'Interrupts Unjam Blocking'),
 ('Build Prototypes', 'Mercenary Analyst'),
 ('Build Prototypes', 'Named Stable Bases'),
 ('Build Prototypes', 'Private World'),
 ('Build Prototypes', 'Programming Episode'),
 ('Build Prototypes', 'Recommitment Meeting '),
 ('Build Prototypes', 'Sacrifice One Person '),
 ('Build Prototypes', 'Size The Schedule'),
 ('Build Prototypes', 'Someone Always Makes Progress '),
 ('Build Prototypes', 'Take No Small Slips'),
 ('Build Prototypes', 'Team Per Task'),
 ('Build Prototypes', 'Wor

In [129]:
df_new = pd.DataFrame(list(combinations(df['pattern_names'].tolist(),2)), columns=['pattern_name1','pattern_name2'])
df_new

,pattern_name1,pattern_name2
0,Build Prototypes,Community Of Trust
1,Build Prototypes,Completion Headroom
2,Build Prototypes,Day Care
3,Build Prototypes,Developer Controls Process
4,Build Prototypes,Development Episode
...,...,...
4181,Standards Linking Locations,Subclass Per Team
4182,Standards Linking Locations,Variation Behind Interface
4183,Stand-Up Meeting,Subclass Per Team
4184,Stand-Up Meeting,Variation Behind Interface


In [130]:
df_new["resulting_embedding1"] = df_new.apply(lambda row: df[df["pattern_names"]==str(row.pattern_name1)].resulting_embedding.item(), axis=1)
df_new["resulting_embedding2"] = df_new.apply(lambda row: df[df["pattern_names"]==str(row.pattern_name2)].resulting_embedding.item(), axis=1)
df_new["starting_embedding1"] = df_new.apply(lambda row: df[df["pattern_names"]==str(row.pattern_name1)].starting_embedding.item(), axis=1)
df_new["starting_embedding2"] = df_new.apply(lambda row: df[df["pattern_names"]==str(row.pattern_name2)].starting_embedding.item(), axis=1)
df_new

,pattern_name1,pattern_name2,resulting_embedding1,resulting_embedding2,starting_embedding1,starting_embedding2
0,Build Prototypes,Community Of Trust,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.013892247, 0.0677837, -0.011958739, 0.01426...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.033992566, 0.0021839319, -0.0064396067, 0.0..."
1,Build Prototypes,Completion Headroom,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[-0.02881442, 0.105386145, -0.0058587473, -0.0...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[-0.06432862, 0.046334974, -0.015850563, -0.08..."
2,Build Prototypes,Day Care,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.071074046, -0.003414135, -0.03895503, 0.012...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.069971845, 0.056567065, -0.00016744515, -0...."
3,Build Prototypes,Developer Controls Process,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.04166916, -0.056860633, -0.026997551, -0.03...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.04957857, 0.05668401, -0.046187162, -0.0080..."
4,Build Prototypes,Development Episode,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.0035809937, 0.0012670304, -0.019009784, 0.0..."
...,...,...,...,...,...,...
4181,Standards Linking Locations,Subclass Per Team,"[0.0061781104, 0.044977684, 0.0036537598, -0.0...","[0.045442764, 0.06214358, -0.015616037, 0.0224...","[0.04809724, -8.491268e-05, -0.025372993, 0.00...","[0.032522, 0.027128672, 0.017792094, 0.0193840..."
4182,Standards Linking Locations,Variation Behind Interface,"[0.0061781104, 0.044977684, 0.0036537598, -0.0...","[-0.003014352, 0.027376406, -0.037597157, 0.02...","[0.04809724, -8.491268e-05, -0.025372993, 0.00...","[0.011085069, 0.045864385, -0.03685276, -0.008..."
4183,Stand-Up Meeting,Subclass Per Team,"[0.021691522, 0.04773192, -0.023035502, 0.0047...","[0.045442764, 0.06214358, -0.015616037, 0.0224...","[0.03001965, 0.053872492, -0.030469585, 0.0029...","[0.032522, 0.027128672, 0.017792094, 0.0193840..."
4184,Stand-Up Meeting,Variation Behind Interface,"[0.021691522, 0.04773192, -0.023035502, 0.0047...","[-0.003014352, 0.027376406, -0.037597157, 0.02...","[0.03001965, 0.053872492, -0.030469585, 0.0029...","[0.011085069, 0.045864385, -0.03685276, -0.008..."


## Skladanie vzorov do sekvencií (nápad)

V tomto bode som skúsil zo sekvencie opísanej v jazyku vzorov ProjectManagementPatternLanguage 
(A Story About Project Management - https://sites.google.com/a/scrumplop.org/published-patterns/Organizational-Patterns-of-Agile-Software-Development/bookoutline/thepatternlanguages/organizationdesignpatterns/projectmanagementpatternlanguage?authuser=0). 


Z príbehu som vybral postupnosti vzorov tak ako išli za sebou. Napríklad prvý je spomínaný vzor BuildPrototypes a následne bol aplikovaný SizeTheSchedule. Takže do tabuľky spravím záznam sequence==1 pre záznam kde sa nachádza tento pár. Keďže som sa chcel zamerať na to kedy, respektíve po aplikovaní akého vzrou je môžné/vhodné aplikovať druhý vzor, bral som tento vzťah len pre dvojicu po sebe idúcich vzorov v príbehu. Teda, po SizeTheSchedule nasleduje vzor NamedStableBases. Dvojica SizeTheSchedule a NamedStableBases budú mať sequence==1, ale pár BuildPrototypes a NamedStableBases bude mať 0 (Nemusí to byť správne uvažovanie). 

In [364]:
df_final["sequence"]=0
df_final

C:\Users\olive\AppData\Local\Temp\ipykernel_4724\4076506384.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["sequence"]=0


,pattern_name1,pattern_name2,resulting_context,starting_context,sequence
1,Build Prototypes,Community Of Trust,Build an isolated prototype solution whose pur...,It is essential that the people in a team trus...,0
2,Build Prototypes,Completion Headroom,Build an isolated prototype solution whose pur...,Every project must commit to delivery on a few...,0
3,Build Prototypes,Day Care,Build an isolated prototype solution whose pur...,Your experts are spending all their time mento...,0
4,Build Prototypes,Developer Controls Process,Build an isolated prototype solution whose pur...,"A development culture, like any culture, can b...",0
5,Build Prototypes,Development Episode,Build an isolated prototype solution whose pur...,It's important to build on the collective stre...,0
...,...,...,...,...,...
670,Work Split,Surrogate Customer,Divide a task into an urgent and deferred comp...,It is important to exchange ideas and clarify ...,0
671,Work Split,Take No Small Slips,Divide a task into an urgent and deferred comp...,It's difficult to know how long a project shou...,0
672,Work Split,Team Per Task,Divide a task into an urgent and deferred comp...,Large distractions (usually called crises) mus...,0
673,Work Split,Work Flows Inward,Divide a task into an urgent and deferred comp...,An organization must seek a structure that bes...,0


In [365]:
#vytvorené dvojice z príbehu
indexes = []

indexes.append(df_final.index[(df_final.pattern_name1=="Build Prototypes") & (df_final.pattern_name2=="Size The Schedule")])
indexes.append(df_final.index[(df_final.pattern_name1=="Size The Schedule") & (df_final.pattern_name2=="Named Stable Bases")])
indexes.append(df_final.index[(df_final.pattern_name1=="Named Stable Bases") & (df_final.pattern_name2=="Private World")])
indexes.append(df_final.index[(df_final.pattern_name1=="Private World") & (df_final.pattern_name2=="Developer Controls Process")])
indexes.append(df_final.index[(df_final.pattern_name1=="Developer Controls Process") & (df_final.pattern_name2=="Work Flows Inward")])
indexes.append(df_final.index[(df_final.pattern_name1=="Developer Controls Process") & (df_final.pattern_name2=="Work Queue")])
indexes.append(df_final.index[(df_final.pattern_name1=="Developer Controls Process") & (df_final.pattern_name2=="Informal Labor Plan")])
indexes.append(df_final.index[(df_final.pattern_name1=="Developer Controls Process") & (df_final.pattern_name2=="Programming Episode")])
indexes.append(df_final.index[(df_final.pattern_name1=="Work Queue") & (df_final.pattern_name2=="Take No Small Slips")])
indexes.append(df_final.index[(df_final.pattern_name1=="Informal Labor Plan") & (df_final.pattern_name2=="Take No Small Slips")])
indexes.append(df_final.index[(df_final.pattern_name1=="Programming Episode") & (df_final.pattern_name2=="Take No Small Slips")])
indexes.append(df_final.index[(df_final.pattern_name1=="Take No Small Slips") & (df_final.pattern_name2=="Recommitment Meeting ")])
indexes.append(df_final.index[(df_final.pattern_name1=="Take No Small Slips") & (df_final.pattern_name2=="Team Per Task")])
indexes.append(df_final.index[(df_final.pattern_name1=="Take No Small Slips") & (df_final.pattern_name2=="Sacrifice One Person ")])
indexes.append(df_final.index[(df_final.pattern_name1=="Team Per Task") & (df_final.pattern_name2=="Someone Always Makes Progress ")])

indexes

[Int64Index([18], dtype='int64'),
 Int64Index([481], dtype='int64'),
 Int64Index([352], dtype='int64'),
 Int64Index([368], dtype='int64'),
 Int64Index([127], dtype='int64'),
 Int64Index([128], dtype='int64'),
 Int64Index([114], dtype='int64'),
 Int64Index([119], dtype='int64'),
 Int64Index([645], dtype='int64'),
 Int64Index([281], dtype='int64'),
 Int64Index([411], dtype='int64'),
 Int64Index([562], dtype='int64'),
 Int64Index([568], dtype='int64'),
 Int64Index([563], dtype='int64'),
 Int64Index([591], dtype='int64')]

In [366]:
for index in indexes:
    df_final.loc[index,'sequence']=1
df_final[df_final['sequence']==1]

,pattern_name1,pattern_name2,resulting_context,starting_context,sequence
18,Build Prototypes,Size The Schedule,Build an isolated prototype solution whose pur...,Both overly ambitious schedules and overly gen...,1
114,Developer Controls Process,Informal Labor Plan,Make the Developer the focal point of process ...,A schedule of developer work tasks can both as...,1
119,Developer Controls Process,Programming Episode,Make the Developer the focal point of process ...,Programming is the act of deciding now what wi...,1
127,Developer Controls Process,Work Flows Inward,Make the Developer the focal point of process ...,An organization must seek a structure that bes...,1
128,Developer Controls Process,Work Queue,Make the Developer the focal point of process ...,"It is difficult to do linear, monochronic sche...",1
281,Informal Labor Plan,Take No Small Slips,Let individuals devise their own short-term pl...,It's difficult to know how long a project shou...,1
352,Named Stable Bases,Private World,Stabilize system interfaces â€” the architectu...,How can we balance the need for developers to ...,1
368,Private World,Developer Controls Process,Provide a mechanism where developers can maint...,"A development culture, like any culture, can b...",1
411,Programming Episode,Take No Small Slips,Develop a program in discrete episodes. Select...,It's difficult to know how long a project shou...,1
481,Size The Schedule,Named Stable Bases,Reward developers for negotiating a schedule t...,It is important to integrate software frequent...,1


## Použitie Neurónovej siete

Zo získaných dát som sa pokúsil spraviť demo neúronovej siete na klasifikáciu dvojíc textov. Teda podobnosť medzi konečným kontextom (resulting_context) vzoru X a začiatočným kontextom (starting_context) vzoru Y. Kde ak tieto kontexy sú podobné, je možné povedať že vzor Y môže byť použitý po vzore X. Zároveň by sieť pracovala s informáciou o tom či už boli v sekvencií čo som robil v prechádzajúcom úseku kódu.

Avšak model, ktorý by dokázal pozerať na sémantiku textov, v mojom prípade to bol BERT respektíve model na ňom založený, dokázal zobrať iba texty do veľkosti 509, čo je príliš málo na moje textové opisy. Existujú riešenia založené na modeli BERT, ktoré zvládnu aj dlhšie texty ako BigBird alebo LongFormer, tie sa mi však nepodarilo spojazdniť zatiaľ. 

In [176]:
max(df_final["starting_context"].apply(len))

1506

In [216]:
val_df["starting_context"].apply(len)

147     324
148    1506
149     232
151     861
152    1346
153     663
154    1003
155    1075
156     492
157     709
158    1281
159     644
160     476
161     324
162    1506
163     232
164     449
166    1346
167     663
168    1003
169    1075
170     492
171     709
172    1281
173     644
174     476
175     324
176    1506
177     232
178     449
179     861
181     663
182    1003
183    1075
184     492
185     709
186    1281
187     644
188     476
189     324
190    1506
191     232
192     449
193     861
194    1346
Name: starting_context, dtype: int64

## Iné riešenie (bez neurónovej siete)

Za použitia BERT modelu all-mpnet-base-v2, ktorý je natrénovaný a pripravený na použitie, viem vytvoriť vektorovú reprezentáciu textu, kde je zachovaná aj jeho sémantika. Vektory oboch kontextov viem porovnávať na základe metrík kosínusová podobnosť alebo euklidovksá vzdialenosť. Tieto mezriky podobností vektorov môžem použiť ako vstupy do ML modelu.

Rátam podobnosť medzi počiatočným a konečným kontextom, oboma začiatočnými kontextami a oboma konečnými kontextami.

In [12]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')


In [131]:
df_final = df_new
df_final

,pattern_name1,pattern_name2,resulting_embedding1,resulting_embedding2,starting_embedding1,starting_embedding2
0,Build Prototypes,Community Of Trust,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.013892247, 0.0677837, -0.011958739, 0.01426...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.033992566, 0.0021839319, -0.0064396067, 0.0..."
1,Build Prototypes,Completion Headroom,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[-0.02881442, 0.105386145, -0.0058587473, -0.0...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[-0.06432862, 0.046334974, -0.015850563, -0.08..."
2,Build Prototypes,Day Care,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.071074046, -0.003414135, -0.03895503, 0.012...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.069971845, 0.056567065, -0.00016744515, -0...."
3,Build Prototypes,Developer Controls Process,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.04166916, -0.056860633, -0.026997551, -0.03...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.04957857, 0.05668401, -0.046187162, -0.0080..."
4,Build Prototypes,Development Episode,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.0035809937, 0.0012670304, -0.019009784, 0.0..."
...,...,...,...,...,...,...
4181,Standards Linking Locations,Subclass Per Team,"[0.0061781104, 0.044977684, 0.0036537598, -0.0...","[0.045442764, 0.06214358, -0.015616037, 0.0224...","[0.04809724, -8.491268e-05, -0.025372993, 0.00...","[0.032522, 0.027128672, 0.017792094, 0.0193840..."
4182,Standards Linking Locations,Variation Behind Interface,"[0.0061781104, 0.044977684, 0.0036537598, -0.0...","[-0.003014352, 0.027376406, -0.037597157, 0.02...","[0.04809724, -8.491268e-05, -0.025372993, 0.00...","[0.011085069, 0.045864385, -0.03685276, -0.008..."
4183,Stand-Up Meeting,Subclass Per Team,"[0.021691522, 0.04773192, -0.023035502, 0.0047...","[0.045442764, 0.06214358, -0.015616037, 0.0224...","[0.03001965, 0.053872492, -0.030469585, 0.0029...","[0.032522, 0.027128672, 0.017792094, 0.0193840..."
4184,Stand-Up Meeting,Variation Behind Interface,"[0.021691522, 0.04773192, -0.023035502, 0.0047...","[-0.003014352, 0.027376406, -0.037597157, 0.02...","[0.03001965, 0.053872492, -0.030469585, 0.0029...","[0.011085069, 0.045864385, -0.03685276, -0.008..."


In [132]:
#počítanie kosínusovej podobnosti

cos_sim1 = []
for pairs in df_final[['resulting_embedding1','starting_embedding2']].values:
    cos_sim1.append(cosine_similarity([pairs[0],pairs[1]]))
cos_values1 = []
for values in cos_sim1:
    cos_values1.append(values[0][1])
    
cos_sim2 = []
for pairs in df_final[['starting_embedding1','starting_embedding2']].values:
    cos_sim2.append(cosine_similarity([pairs[0],pairs[1]]))
cos_values2 = []
for values in cos_sim2:
    cos_values2.append(values[0][1])
    
cos_sim3 = []
for pairs in df_final[['resulting_embedding1','resulting_embedding2']].values:
    cos_sim3.append(cosine_similarity([pairs[0],pairs[1]]))
cos_values3 = []
for values in cos_sim3:
    cos_values3.append(values[0][1])
    
cos_sim4 = []
for pairs in df_final[['starting_embedding1','resulting_embedding2']].values:
    cos_sim4.append(cosine_similarity([pairs[0],pairs[1]]))
cos_values4 = []
for values in cos_sim4:
    cos_values4.append(values[0][1])

In [133]:
df_final['cos_similarity_result_start_1']=cos_values1
df_final['cos_similarity_start_start']=cos_values2
df_final['cos_similarity_result_result']=cos_values3
df_final['cos_similarity_result_start_2']=cos_values4

In [409]:
df_final['euc_distance']=euc_values

C:\Users\olive\AppData\Local\Temp\ipykernel_4724\2520422237.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['euc_distance']=euc_values


Výsledná tabuľka s metrikami

In [134]:
df_final

,pattern_name1,pattern_name2,resulting_embedding1,resulting_embedding2,starting_embedding1,starting_embedding2,cos_similarity_result_start_1,cos_similarity_start_start,cos_similarity_result_result,cos_similarity_result_start_2
0,Build Prototypes,Community Of Trust,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.013892247, 0.0677837, -0.011958739, 0.01426...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.033992566, 0.0021839319, -0.0064396067, 0.0...",0.252579,0.399197,0.249860,0.350796
1,Build Prototypes,Completion Headroom,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[-0.02881442, 0.105386145, -0.0058587473, -0.0...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[-0.06432862, 0.046334974, -0.015850563, -0.08...",0.147638,0.287624,0.218520,0.267191
2,Build Prototypes,Day Care,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.071074046, -0.003414135, -0.03895503, 0.012...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.069971845, 0.056567065, -0.00016744515, -0....",0.224507,0.230168,0.334633,0.230830
3,Build Prototypes,Developer Controls Process,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.04166916, -0.056860633, -0.026997551, -0.03...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.04957857, 0.05668401, -0.046187162, -0.0080...",0.277095,0.514923,0.239971,0.340766
4,Build Prototypes,Development Episode,"[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...",0.064959,0.175616,0.341376,0.349847
...,...,...,...,...,...,...,...,...,...,...
4181,Standards Linking Locations,Subclass Per Team,"[0.0061781104, 0.044977684, 0.0036537598, -0.0...","[0.045442764, 0.06214358, -0.015616037, 0.0224...","[0.04809724, -8.491268e-05, -0.025372993, 0.00...","[0.032522, 0.027128672, 0.017792094, 0.0193840...",0.316697,0.326791,0.456721,0.418585
4182,Standards Linking Locations,Variation Behind Interface,"[0.0061781104, 0.044977684, 0.0036537598, -0.0...","[-0.003014352, 0.027376406, -0.037597157, 0.02...","[0.04809724, -8.491268e-05, -0.025372993, 0.00...","[0.011085069, 0.045864385, -0.03685276, -0.008...",0.303342,0.374386,0.384451,0.260962
4183,Stand-Up Meeting,Subclass Per Team,"[0.021691522, 0.04773192, -0.023035502, 0.0047...","[0.045442764, 0.06214358, -0.015616037, 0.0224...","[0.03001965, 0.053872492, -0.030469585, 0.0029...","[0.032522, 0.027128672, 0.017792094, 0.0193840...",0.206600,0.336444,0.319453,0.424053
4184,Stand-Up Meeting,Variation Behind Interface,"[0.021691522, 0.04773192, -0.023035502, 0.0047...","[-0.003014352, 0.027376406, -0.037597157, 0.02...","[0.03001965, 0.053872492, -0.030469585, 0.0029...","[0.011085069, 0.045864385, -0.03685276, -0.008...",0.300970,0.429855,0.243869,0.359017


## Podobnosti konečného kontextu vzoru X a začiatočného vzoru Y
Najväčšiu podobnosť mali vzory CompletionHeadRoom a WorkSplit čo keď sa pozrieme do ích textových opisov tak je vidieť, že tam naozaj existuje nejaký vzťah.

Po pridaní ďalšieho jazyka vzorov vznikol ďalšia silná podobnosť medzi EngageCustomers a SurrogateCustomer.

In [86]:
df_final.sort_values(by=['cos_similarity_result_start_1'],ascending=False).head(20)

,pattern_name1,pattern_name2,resulting_embedding1,resulting_embedding2,starting_embedding1,starting_embedding2,cos_similarity_result_start_1,cos_similarity_start_start,cos_similarity_result_result,cos_similarity_result_start_2,result_start_1,result_start_2,start_start,result_result
2374,EngageCustomers,Surrogate Customer,"[0.087259285, 0.014686615, -0.046560932, 0.028...","[0.039613444, 0.06593218, -0.027706152, 0.0065...","[0.08553004, 0.04444293, -0.029072814, -0.0267...","[0.07129714, 0.05089531, -0.022737782, 0.02519...",0.754471,0.771849,0.695268,0.652926,1,1,1,1
3534,Coupling Decreases Latency,Shaping Circulation Realms,"[0.050980132, -0.043490957, -0.041003358, -0.0...","[0.05007569, 0.063153945, -0.029606206, 0.0309...","[0.058342695, 0.034285374, -0.046130203, -0.02...","[0.11800907, -0.0045809764, -0.047178406, 0.02...",0.734574,0.524197,0.570776,0.376426,1,0,0,0
202,Completion Headroom,Work Split,"[-0.02881442, 0.105386145, -0.0058587473, -0.0...","[0.0035704507, 0.05868317, -0.023359573, -0.02...","[-0.06432862, 0.046334974, -0.015850563, -0.08...","[0.015930163, 0.03433592, -0.0064756554, -0.05...",0.727226,0.311419,0.454712,0.391157,1,0,0,0
2419,Engage Quality Assurance,Group Validation,"[0.032119405, 0.05401316, -0.026115092, 0.0091...","[0.04413286, 0.012007737, -0.014635134, -0.007...","[0.050087288, 0.05367348, -0.05450759, 0.00260...","[0.043183528, 0.023295484, -0.037826177, 0.007...",0.705157,0.618322,0.646996,0.632082,1,1,1,1
4086,Architecture Team,LockEmUpTogether,"[0.05740328, -0.02206143, -0.041312918, 0.0129...","[-0.0125033455, 0.061438777, -0.0067345113, 0....","[0.028231397, 0.097609654, -0.009499119, 0.038...","[0.032803465, 0.02584359, -0.016275993, 0.0020...",0.704501,0.746565,0.056356,-0.013624,1,0,1,0
3697,Few Roles,Producer Roles,"[0.039559744, 0.040482394, -0.0056656487, 0.01...","[0.043406006, 0.039052162, -0.009858476, 0.035...","[0.07127675, 0.017844955, -0.038920935, 0.0096...","[0.042521015, 0.07876812, -0.010715093, 0.0356...",0.697790,0.513453,0.771388,0.550869,1,0,0,1
3603,Distribute Work Evenly,Shaping Circulation Realms,"[0.061786447, 0.03943652, -0.031095436, 0.0665...","[0.05007569, 0.063153945, -0.029606206, 0.0309...","[0.048017904, -0.04110071, -0.04062479, -0.024...","[0.11800907, -0.0045809764, -0.047178406, 0.02...",0.689090,0.716949,0.638686,0.475153,1,0,1,1
3521,Coupling Decreases Latency,Distribute Work Evenly,"[0.050980132, -0.043490957, -0.041003358, -0.0...","[0.061786447, 0.03943652, -0.031095436, 0.0665...","[0.058342695, 0.034285374, -0.046130203, -0.02...","[0.048017904, -0.04110071, -0.04062479, -0.024...",0.683988,0.539114,0.639402,0.485481,1,0,0,1
2248,Diverse Groups,Self-Selecting Team,"[0.024144419, 0.022501651, -0.011508377, 0.014...","[0.022171535, 0.044167697, -0.03011777, 0.0415...","[0.050589025, 0.024854558, -0.020808082, -0.01...","[0.033132505, 0.029942734, -0.041641798, 0.013...",0.678630,0.636739,0.652056,0.435856,1,0,1,1
3524,Coupling Decreases Latency,Few Roles,"[0.050980132, -0.043490957, -0.041003358, -0.0...","[0.039559744, 0.040482394, -0.0056656487, 0.01...","[0.058342695, 0.034285374, -0.046130203, -0.02...","[0.07127675, 0.017844955, -0.038920935, 0.0096...",0.676673,0.724188,0.512609,0.480308,1,0,1,0


Naopak najmenej podobné kontexty mali MercenaryAnalyst a NamedStableBases.

Po pridaní ďalšieho jazyka vzorov vznikol ďalšia veľmi slabá podobnosť medzi InterruptsUnjamBlocking a SoloVirtuoso.

In [153]:
df_final.sort_values(by=['cos_similarity_result_start_1']).head(10)

,pattern_name1,pattern_name2,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,starting_embedding2,resulting_embedding2,cos_similarity_result_start_1,cos_similarity_start_start,cos_similarity_result_result,cos_similarity_result_start_2
564,Interrupts Unjam Blocking,Solo Virtuoso,If a role is about to block on a critical reso...,Do the entire design and implementation with o...,A comprehensive scheduling plan is difficult i...,When a smaller software project (less than twe...,"[-0.0021273368, 0.07085707, -0.023154281, -0.0...","[0.009520725, -0.01611056, -0.0016354743, -0.0...","[0.011190042, 0.03326897, -0.012879533, -0.068...","[0.042240884, 0.066174544, -0.028253678, 0.019...",-0.010899,0.353839,0.199666,0.273364
885,Take No Small Slips,Group Validation,Prefer a single large slip to several small sl...,"Even before engaging QA, the development team ...",It's difficult to know how long a project shou...,Product quality is crucial to the success of t...,"[0.015812408, 0.10102768, -0.009773148, -0.043...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.043183528, 0.023295484, -0.037826177, 0.007...","[0.04413286, 0.012007737, -0.014635134, -0.007...",-0.009680,0.212549,0.019112,0.294899
772,Sacrifice One Person,Application Design Is Bounded By Test Design,Assign just one person to it until it gets han...,Use case-driven test design starts when the cu...,"Small distractions can add up, and sap the str...",When do you design and implement test plans an...,"[0.010107814, -0.04778266, -0.017414702, -0.01...","[0.00020712917, -0.003472787, -0.018456511, 0....","[-0.022668341, -0.03599587, -0.019662376, -0.0...","[0.022802858, -0.008028176, -0.03563411, -0.01...",-0.006683,0.012333,0.016948,0.025686
1052,Application Design Is Bounded By Test Design,Wise Fool,Use case-driven test design starts when the cu...,"Nurture the role of the wise fool, who can rai...",When do you design and implement test plans an...,Interpersonal dynamics often discourage good i...,"[-0.022668341, -0.03599587, -0.019662376, -0.0...","[0.022802858, -0.008028176, -0.03563411, -0.01...","[0.03659603, 0.013783946, 0.027074216, 0.01551...","[0.04201864, 0.072648905, 0.027401034, 0.02183...",-0.005077,0.132434,0.100973,0.105599
1059,Apprenticeship,Failed Project Wake,Turn new hires into experts (see DomainExperti...,Hold a wake for the failed project. This shoul...,A project must balance its need for growth wit...,"Canceling a project, even for the best externa...","[0.03503278, 0.026869062, -0.012360971, -0.025...","[0.0432205, 0.04235784, 0.0030211227, -0.03399...","[-0.0326711, 0.05866426, 0.010470345, -0.00173...","[-0.038515694, 0.13793832, 0.009984291, 0.0018...",0.000786,0.303404,0.072112,0.285142
882,Take No Small Slips,Failed Project Wake,Prefer a single large slip to several small sl...,Hold a wake for the failed project. This shoul...,It's difficult to know how long a project shou...,"Canceling a project, even for the best externa...","[0.015812408, 0.10102768, -0.009773148, -0.043...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[-0.0326711, 0.05866426, 0.010470345, -0.00173...","[-0.038515694, 0.13793832, 0.009984291, 0.0018...",0.001967,0.379410,0.046117,0.265437
1032,Application Design Is Bounded By Test Design,Failed Project Wake,Use case-driven test design starts when the cu...,Hold a wake for the failed project. This shoul...,When do you design and implement test plans an...,"Canceling a project, even for the best externa...","[-0.022668341, -0.03599587, -0.019662376, -0.0...","[0.022802858, -0.008028176, -0.03563411, -0.01...","[-0.0326711, 0.05866426, 0.010470345, -0.00173...","[-0.038515694, 0.13793832, 0.009984291, 0.0018...",0.005775,0.006782,0.067180,0.100044
570,Mercenary Analyst,Named Stable Bases,"Hire a technical writer, proficient in the nec...",Stabilize system interfaces â€” the architectu...,Technical documentation is the dirty work ever...,It is importa

## Podobnosti konečného kontextu vzoru Y a začiatočného vzoru X
Najväčšiu podobnosť mali vzory DeveloperControllsProcess a WorkFlowsInward.

In [194]:
df_final.sort_values(by=['cos_similarity_result_start_2'],ascending=False).head(10)

,pattern_name1,pattern_name2,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,starting_embedding2,resulting_embedding2,cos_similarity_result_start_1,cos_similarity_start_start,cos_similarity_result_result,cos_similarity_result_start_2,result_start_1,result_start_2,start_start,result_result
223,Developer Controls Process,Work Flows Inward,Make the Developer the focal point of process ...,Work should flow in to developer from stakehol...,"A development culture, like any culture, can b...",An organization must seek a structure that bes...,"[0.04957857, 0.05668401, -0.046187162, -0.0080...","[0.04166916, -0.056860633, -0.026997551, -0.03...","[0.04887739, 0.0011259597, -0.029870065, -0.04...","[0.03404403, 0.020410828, -0.044784002, 0.0222...",0.509857,0.686973,0.470207,0.655913,0,1,1,0
1196,EngageCustomers,urrogate Customer,Closely couple the Customer role to the Develo...,Create a SurrogateCustomer role in the project...,It's important that the development organizati...,It is important to exchange ideas and clarify ...,"[0.08553004, 0.04444293, -0.029072814, -0.0267...","[0.087259285, 0.014686615, -0.046560932, 0.028...","[0.07129714, 0.05089531, -0.022737782, 0.02519...","[0.039613444, 0.06593218, -0.027706152, 0.0065...",0.754471,0.771849,0.695268,0.652926,1,1,1,1
1056,Apprenticeship,Domain Expertise In Roles,Turn new hires into experts (see DomainExperti...,"Hire domain experts with proven track records,...",A project must balance its need for growth wit...,Matching staff with roles is one of the hardes...,"[0.03503278, 0.026869062, -0.012360971, -0.025...","[0.0432205, 0.04235784, 0.0030211227, -0.03399...","[0.033357415, 0.02620629, -0.018051252, -0.044...","[0.06965132, 0.02252271, -0.03190883, 0.007860...",0.361728,0.605948,0.530066,0.652040,0,1,1,0
1203,Engage Quality Assurance,Group Validation,Make QA a central role. Couple it tightly to d...,"Even before engaging QA, the development team ...",Customer engagement is a key element of qualit...,Product quality is crucial to the success of t...,"[0.050087288, 0.05367348, -0.05450759, 0.00260...","[0.032119405, 0.05401316, -0.026115092, 0.0091...","[0.043183528, 0.023295484, -0.037826177, 0.007...","[0.04413286, 0.012007737, -0.014635134, -0.007...",0.705157,0.618322,0.646996,0.632082,1,1,1,1
278,Development Episode,Diverse Groups,Approach all development as a group activity a...,Consider temperaments and diverse experience b...,It's important to build on the collective stre...,Homogeneous teams that comprise too many of th...,"[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.050589025, 0.024854558, -0.020808082, -0.01...","[0.024144419, 0.022501651, -0.011508377, 0.014...",0.577250,0.485816,0.552055,0.620068,0,1,0,0
1428,Team Pride,Unity Of Purpose,"""We're the best."" Instill a sense of elitism i...",The leader of the project must instill a commo...,People are most successful when they feel good...,Many projects have rocky beginnings as people ...,"[0.00018179686, 0.04582352, 0.001021349, 0.009...","[0.024695326, 0.011348519, 0.022361487, 0.0477...","[0.03618246, 0.029766077, -0.025872428, -1.552...","[0.05665192, 0.033690587, 0.00075172447, 0.032...",0.393009,0.576701,0.658926,0.611281,0,1,0,1
532,Interrupts Unjam Blocking,Recommitment Meeting,If a role is about to block on a critical reso...,Assemble a meeting of interested management an...,A comprehensive scheduling plan is difficult i...,If a Product Initiative is in jeopardy because...,"[-0.0021273368, 0.07085707, -0.023154281, -0.0...","[0.009520725, -0.01611056, -0.0016354743, -0.0...","[0.010786134, 0.10886827, 0.003984304, -0.0595...","[-0.011295064, 0.060159247, -0.019415945, -0.0...",0.371767,0.309565,0.261467,0.598939,0,0,0,0
43,Build Prototypes,Scenarios Define Problem,Build an isolated prototype solution whose pur...,Capture system functional requirements as use ...,A project must test 

Naopak najmenej podobné kontexty mali ApplicationDesignIsBoundedByTestDesign a PatronRole.

In [195]:
df_final.sort_values(by=['cos_similarity_result_start_2']).head(10)

,pattern_name1,pattern_name2,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,starting_embedding2,resulting_embedding2,cos_similarity_result_start_1,cos_similarity_start_start,cos_similarity_result_result,cos_similarity_result_start_2,result_start_1,result_start_2,start_start,result_result
1040,Application Design Is Bounded By Test Design,Patron Role,Use case-driven test design starts when the cu...,"Give the project access to a visible, high-lev...",When do you design and implement test plans an...,It is important to give a project continuity. ...,"[-0.022668341, -0.03599587, -0.019662376, -0.0...","[0.022802858, -0.008028176, -0.03563411, -0.01...","[0.0020334988, 0.09311888, -0.009235093, -0.02...","[0.021286698, 0.051788032, 0.0002978026, 0.053...",0.081237,0.164928,0.066896,-0.003953,0,0,0,0
1236,Failed Project Wake,Subsystem By Skill,Hold a wake for the failed project. This shoul...,Separate subsystems by staff skills and skill ...,"Canceling a project, even for the best externa...",Birds of a feather flock together. By ConwaysL...,"[-0.0326711, 0.05866426, 0.010470345, -0.00173...","[-0.038515694, 0.13793832, 0.009984291, 0.0018...","[0.0250982, 0.033371083, -0.0030294019, -0.012...","[0.019256314, -0.011144504, -0.041393787, 0.00...",0.098575,0.102388,0.064086,0.013852,0,0,0,0
993,Work Queue,Team Pride,Produce a schedule that is simply a prioritize...,"""We're the best."" Instill a sense of elitism i...","It is difficult to do linear, monochronic sche...",People are most successful when they feel good...,"[0.023785634, 0.08178888, -0.02493737, -0.0035...","[0.023706136, 0.051402166, -0.029743375, -0.00...","[0.00018179686, 0.04582352, 0.001021349, 0.009...","[0.024695326, 0.011348519, 0.022361487, 0.0477...",0.217191,0.194132,0.109805,0.023218,0,0,0,0
772,Sacrifice One Person,Application Design Is Bounded By Test Design,Assign just one person to it until it gets han...,Use case-driven test design starts when the cu...,"Small distractions can add up, and sap the str...",When do you design and implement test plans an...,"[0.010107814, -0.04778266, -0.017414702, -0.01...","[0.00020712917, -0.003472787, -0.018456511, 0....","[-0.022668341, -0.03599587, -0.019662376, -0.0...","[0.022802858, -0.008028176, -0.03563411, -0.01...",-0.006683,0.012333,0.016948,0.025686,0,0,0,0
16,Build Prototypes,Sacrifice One Person,Build an isolated prototype solution whose pur...,Assign just one person to it until it gets han...,A project must test requirements and design de...,"Small distractions can add up, and sap the str...","[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[0.010107814, -0.04778266, -0.017414702, -0.01...","[0.00020712917, -0.003472787, -0.018456511, 0....",0.052221,0.141461,0.111258,0.025786,0,0,0,0
1034,Application Design Is Bounded By Test Design,Gate Keeper,Use case-driven test design starts when the cu...,"One project member, a PublicCharacter with an ...",When do you design and implement test plans an...,A project must develop good interfaces with th...,"[-0.022668341, -0.03599587, -0.019662376, -0.0...","[0.022802858, -0.008028176, -0.03563411, -0.01...","[0.0013977767, 0.061598483, -0.021921508, 0.00...","[0.036360897, 0.07367269, -0.0060292576, 0.013...",0.163675,0.228657,0.113079,0.026636,0,0,0,0
995,Work Queue,Wise Fool,Produce a schedule that is simply a prioritize...,"Nurture the role of the wise fool, who can rai...","It is difficult to do linear, monochronic sche...",Interpersonal dynamics often discourage good i...,"[0.023785634, 0.08178888, -0.02493737, -0.0035...","[0.023706136, 0.051402166, -0.029743375, -0.00...","[0.03659603, 0.013783946, 0.027074216, 0.01551...","[0.04201864, 0.072648905, 0.027401034, 0.02183...",0.138955,0.084831,0.151450,0.027779,0,0,0,0
177,Day Care,Application Design Is Bounded By Test Design,"Put one expert in charge of all the novices, l...",Use case-driven t

## Podobnosti začiatočného kontextu vzoru X a začiatočného vzoru Y


In [32]:
df_final.sort_values(by=['cos_similarity_start_start']).head(10)

,pattern_name1,pattern_name2,resulting_context,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,resulting_embedding2,starting_embedding2,cos_similarity_result_start,cos_similarity_start_start,cos_similarity_result_result
391,Programming Episode,Community Of Trust,Develop a program in discrete episodes. Select...,Develop a program in discrete episodes. Select...,"Do things that explicitly demonstrate trust, s...",Programming is the act of deciding now what wi...,It is essential that the people in a team trus...,"[0.034314055, 0.04872385, -0.014165695, -0.047...","[0.013992956, 0.089137785, -0.031594567, -0.04...","[0.013892247, 0.0677837, -0.011958739, 0.01426...","[0.033992566, 0.0021839319, -0.0064396067, 0.0...",0.118411,0.086439,0.222910
41,Community Of Trust,Programming Episode,"Do things that explicitly demonstrate trust, s...","Do things that explicitly demonstrate trust, s...",Develop a program in discrete episodes. Select...,It is essential that the people in a team trus...,Programming is the act of deciding now what wi...,"[0.033992566, 0.0021839319, -0.0064396067, 0.0...","[0.013892247, 0.0677837, -0.011958739, 0.01426...","[0.013992956, 0.089137785, -0.031594567, -0.04...","[0.034314055, 0.04872385, -0.014165695, -0.047...",0.190853,0.086439,0.222910
132,Development Episode,Completion Headroom,Approach all development as a group activity a...,Approach all development as a group activity a...,Project work group completion dates from remai...,It's important to build on the collective stre...,Every project must commit to delivery on a few...,"[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[-0.02881442, 0.105386145, -0.0058587473, -0.0...","[-0.06432862, 0.046334974, -0.015850563, -0.08...",0.145468,0.090668,0.393857
57,Completion Headroom,Development Episode,Project work group completion dates from remai...,Project work group completion dates from remai...,Approach all development as a group activity a...,Every project must commit to delivery on a few...,It's important to build on the collective stre...,"[-0.06432862, 0.046334974, -0.015850563, -0.08...","[-0.02881442, 0.105386145, -0.0058587473, -0.0...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...",0.237150,0.090668,0.393857
154,Development Episode,Work Queue,Approach all development as a group activity a...,Approach all development as a group activity a...,Produce a schedule that is simply a prioritize...,It's important to build on the collective stre...,"It is difficult to do linear, monochronic sche...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.023706136, 0.051402166, -0.029743375, -0.00...","[0.023785634, 0.08178888, -0.02493737, -0.0035...",0.205864,0.094205,0.277859
629,Work Queue,Development Episode,Produce a schedule that is simply a prioritize...,Produce a schedule that is simply a prioritize...,Approach all development as a group activity a...,"It is difficult to do linear, monochronic sche...",It's important to build on the collective stre...,"[0.023785634, 0.08178888, -0.02493737, -0.0035...","[0.023706136, 0.051402166, -0.029743375, -0.00...","[0.048495773, 0.012811256, -0.018782204, -0.00...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...",0.161577,0.094205,0.277859
627,Work Queue,Day Care,Produce a schedule that is simply a prioritize...,Produce a schedule that is simply a prioritize...,"Put one expert in charge of all the novices, l...","It is difficult to do linear, monochronic sche...",Your experts are spending all their time mento...,"[0.023785634, 0.08178888, -0.02493737, -0.0035...","[0.023706136, 0.051402166, -0.029743375, -0.00...","[0.071074046, -0.003414135, -0.03895503, 0.012...","[0.069971845, 0.056567065, -0.00016744515, -0....",0.180253,0.118957,0.261723
102,Day Care,Work Queue,"Put one exper

In [117]:
df_final.sort_values(by=['cos_similarity_start_start'],ascending=False).head(10)

,pattern_name1,pattern_name2,resulting_embedding1,resulting_embedding2,starting_embedding1,starting_embedding2,cos_similarity_result_start_1,cos_similarity_start_start,cos_similarity_result_result,cos_similarity_result_start_2,result_start_1,result_start_2,start_start,result_result
3686,Face To Face Before Working Remotely,Standards Linking Locations,"[0.021121738, 0.052539922, -0.008850389, -0.00...","[0.0061781104, 0.044977684, 0.0036537598, -0.0...","[0.021847326, 0.05386007, -0.026924813, -0.020...","[0.04809724, -8.491268e-05, -0.025372993, 0.00...",0.644602,0.847560,0.408057,0.580849,1,0,1,0
3857,Organization Follows Location,Standards Linking Locations,"[0.012706029, 0.019229218, -0.027521659, 0.006...","[0.0061781104, 0.044977684, 0.0036537598, -0.0...","[0.041519586, 0.01019618, -0.04016941, -0.0383...","[0.04809724, -8.491268e-05, -0.025372993, 0.00...",0.560899,0.812614,0.529320,0.529077,0,0,1,0
3152,Size The Organization,Solo Virtuoso,"[0.037290186, -0.0263837, -0.016910883, -0.052...","[0.042240884, 0.066174544, -0.028253678, 0.019...","[0.049533434, 0.028160773, -0.027851027, -0.05...","[0.011190042, 0.03326897, -0.012879533, -0.068...",0.468928,0.796201,0.447995,0.422041,0,0,1,0
3663,Face To Face Before Working Remotely,Organization Follows Location,"[0.021121738, 0.052539922, -0.008850389, -0.00...","[0.012706029, 0.019229218, -0.027521659, 0.006...","[0.021847326, 0.05386007, -0.026924813, -0.020...","[0.041519586, 0.01019618, -0.04016941, -0.0383...",0.562793,0.790836,0.309217,0.592155,0,0,1,0
1817,Work Flows Inward,Responsibilities Engage,"[0.03404403, 0.020410828, -0.044784002, 0.0222...","[-0.016126256, 0.043943252, -0.03567842, 0.022...","[0.04887739, 0.0011259597, -0.029870065, -0.04...","[0.020769138, 0.016824787, -0.02429739, -0.011...",0.584572,0.780409,0.382884,0.499975,0,0,1,0
2374,EngageCustomers,Surrogate Customer,"[0.087259285, 0.014686615, -0.046560932, 0.028...","[0.039613444, 0.06593218, -0.027706152, 0.0065...","[0.08553004, 0.04444293, -0.029072814, -0.0267...","[0.07129714, 0.05089531, -0.022737782, 0.02519...",0.754471,0.771849,0.695268,0.652926,1,1,1,1
1486,Size The Schedule,Take No Small Slips,"[0.020484738, 0.051352836, -0.035016783, -0.00...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.020185143, 0.10455826, -0.008474356, -0.041...","[0.015812408, 0.10102768, -0.009773148, -0.043...",0.555571,0.770337,0.279796,0.394989,0,0,1,0
2056,Apprenticeship,Phasing It In,"[0.0432205, 0.04235784, 0.0030211227, -0.03399...","[0.04751829, 0.05801926, -0.01015259, -0.04318...","[0.03503278, 0.026869062, -0.012360971, -0.025...","[0.018057844, 0.00972775, 0.0013048564, -0.058...",0.313999,0.770281,0.488309,0.574087,0,0,1,0
4050,Architect Also Implements,Architect Controls Product,"[0.04277064, 0.036637824, -0.058282424, 0.0248...","[0.041867714, 0.05750634, -0.040326852, 0.0297...","[0.04869245, 0.09208217, -0.038597852, 0.01699...","[0.042300753, 0.037461042, -0.0526234, 0.00941...",0.517057,0.767638,0.744609,0.577643,0,0,1,1
2533,Fire Walls,Gate Keeper,"[0.028908951, 0.05352523, 0.0076098167, 0.0191...","[0.036360897, 0.07367269, -0.0060292576, 0.013...","[0.012233101, 0.068203, -0.033718087, 0.039389...","[0.0013977767, 0.061598483, -0.021921508, 0.00...",0.433470,0.767022,0.448159,0.452868,0,0,1,0


## Podobnosti konečného kontextu vzoru X a konečného vzoru Y


In [135]:
df_final.sort_values(by=['cos_similarity_result_result'],ascending=False).head(10)

,pattern_name1,pattern_name2,resulting_embedding1,resulting_embedding2,starting_embedding1,starting_embedding2,cos_similarity_result_start_1,cos_similarity_start_start,cos_similarity_result_result,cos_similarity_result_start_2
3697,Few Roles,Producer Roles,"[0.039559744, 0.040482394, -0.0056656487, 0.01...","[0.043406006, 0.039052162, -0.009858476, 0.035...","[0.07127675, 0.017844955, -0.038920935, 0.0096...","[0.042521015, 0.07876812, -0.010715093, 0.0356...",0.697790,0.513453,0.771388,0.550869
4050,Architect Also Implements,Architect Controls Product,"[0.04277064, 0.036637824, -0.058282424, 0.0248...","[0.041867714, 0.05750634, -0.040326852, 0.0297...","[0.04869245, 0.09208217, -0.038597852, 0.01699...","[0.042300753, 0.037461042, -0.0526234, 0.00941...",0.517057,0.767638,0.744609,0.577643
433,Developer Controls Process,Feature Assignment,"[0.04166916, -0.056860633, -0.026997551, -0.03...","[0.063425906, 0.05111946, -0.02904057, 0.00432...","[0.04957857, 0.05668401, -0.046187162, -0.0080...","[0.010199863, 0.08351122, -0.040354326, 0.0039...",0.564871,0.510738,0.734594,0.555362
3626,Divide And Conquer,Few Roles,"[0.024713801, 0.079780094, -0.020676563, -0.03...","[0.039559744, 0.040482394, -0.0056656487, 0.01...","[0.0052802106, 0.09707901, -0.0042288643, -0.0...","[0.07127675, 0.017844955, -0.038920935, 0.0096...",0.461964,0.465399,0.731958,0.663376
4155,LockEmUpTogether,Stand-Up Meeting,"[0.021650521, 0.055338625, -0.0053595942, 0.02...","[0.021691522, 0.04773192, -0.023035502, 0.0047...","[0.01839559, 0.052684497, -0.02552097, 0.00955...","[0.03001965, 0.053872492, -0.030469585, 0.0029...",0.670891,0.576384,0.711472,0.470388
3812,Move Responsibilities,Responsibilities Engage,"[0.026626717, 0.057851892, -0.011990199, 0.020...","[-0.016126256, 0.043943252, -0.03567842, 0.022...","[0.019582674, 0.04502303, -0.0052302727, 0.001...","[0.020769138, 0.016824787, -0.02429739, -0.011...",0.415789,0.540125,0.705553,0.548541
2374,EngageCustomers,Surrogate Customer,"[0.087259285, 0.014686615, -0.046560932, 0.028...","[0.039613444, 0.06593218, -0.027706152, 0.0065...","[0.08553004, 0.04444293, -0.029072814, -0.0267...","[0.07129714, 0.05089531, -0.022737782, 0.02519...",0.754471,0.771849,0.695268,0.652926
449,Development Episode,Informal Labor Plan,"[0.048495773, 0.012811256, -0.018782204, -0.00...","[-0.018673072, 0.08835107, -0.023630276, -0.00...","[0.0035809937, 0.0012670304, -0.019009784, 0.0...","[0.017129539, 0.06161713, -0.04943146, -0.0459...",0.384728,0.272391,0.689016,0.383551
3811,Move Responsibilities,Producer Roles,"[0.026626717, 0.057851892, -0.011990199, 0.020...","[0.043406006, 0.039052162, -0.009858476, 0.035...","[0.019582674, 0.04502303, -0.0052302727, 0.001...","[0.042521015, 0.07876812, -0.010715093, 0.0356...",0.585648,0.625703,0.687694,0.576743
3640,Divide And Conquer,Upside-Down Matrix Management,"[0.024713801, 0.079780094, -0.020676563, -0.03...","[0.0690434, 0.059755735, -0.014117091, 0.02051...","[0.0052802106, 0.09707901, -0.0042288643, -0.0...","[0.045307226, 0.05413821, -0.0041761557, -0.02...",0.558804,0.603705,0.682296,0.551489


In [34]:
df_final.sort_values(by=['cos_similarity_result_result']).head(10)

,pattern_name1,pattern_name2,resulting_context,resulting_context1,resulting_context2,starting_context1,starting_context2,starting_embedding1,resulting_embedding1,resulting_embedding2,starting_embedding2,cos_similarity_result_start,cos_similarity_start_start,cos_similarity_result_result
549,Take No Small Slips,Day Care,Prefer a single large slip to several small sl...,Prefer a single large slip to several small sl...,"Put one expert in charge of all the novices, l...",It's difficult to know how long a project shou...,Your experts are spending all their time mento...,"[0.015812408, 0.10102768, -0.009773148, -0.043...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.071074046, -0.003414135, -0.03895503, 0.012...","[0.069971845, 0.056567065, -0.00016744515, -0....",0.049555,0.361423,-0.027780
99,Day Care,Take No Small Slips,"Put one expert in charge of all the novices, l...","Put one expert in charge of all the novices, l...",Prefer a single large slip to several small sl...,Your experts are spending all their time mento...,It's difficult to know how long a project shou...,"[0.069971845, 0.056567065, -0.00016744515, -0....","[0.071074046, -0.003414135, -0.03895503, 0.012...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.015812408, 0.10102768, -0.009773148, -0.043...",0.179358,0.361423,-0.027780
566,Take No Small Slips,Surrogate Customer,Prefer a single large slip to several small sl...,Prefer a single large slip to several small sl...,Create a SurrogateCustomer role in the project...,It's difficult to know how long a project shou...,It is important to exchange ideas and clarify ...,"[0.015812408, 0.10102768, -0.009773148, -0.043...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.039613444, 0.06593218, -0.027706152, 0.0065...","[0.07129714, 0.05089531, -0.022737782, 0.02519...",0.029477,0.412247,-0.010316
541,Surrogate Customer,Take No Small Slips,Create a SurrogateCustomer role in the project...,Create a SurrogateCustomer role in the project...,Prefer a single large slip to several small sl...,It is important to exchange ideas and clarify ...,It's difficult to know how long a project shou...,"[0.07129714, 0.05089531, -0.022737782, 0.02519...","[0.039613444, 0.06593218, -0.027706152, 0.0065...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.015812408, 0.10102768, -0.009773148, -0.043...",0.266741,0.412247,-0.010316
125,Developer Controls Process,Take No Small Slips,Make the Developer the focal point of process ...,Make the Developer the focal point of process ...,Prefer a single large slip to several small sl...,"A development culture, like any culture, can b...",It's difficult to know how long a project shou...,"[0.04957857, 0.05668401, -0.046187162, -0.0080...","[0.04166916, -0.056860633, -0.026997551, -0.03...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.015812408, 0.10102768, -0.009773148, -0.043...",0.255678,0.320126,0.039159
550,Take No Small Slips,Developer Controls Process,Prefer a single large slip to several small sl...,Prefer a single large slip to several small sl...,Make the Developer the focal point of process ...,It's difficult to know how long a project shou...,"A development culture, like any culture, can b...","[0.015812408, 0.10102768, -0.009773148, -0.043...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.04166916, -0.056860633, -0.026997551, -0.03...","[0.04957857, 0.05668401, -0.046187162, -0.0080...",0.098928,0.320126,0.039159
21,Build Prototypes,Take No Small Slips,Build an isolated prototype solution whose pur...,Build an isolated prototype solution whose pur...,Prefer a single large slip to several small sl...,A project must test requirements and design de...,It's difficult to know how long a project shou...,"[0.05032039, 0.019018563, 0.0053541255, -0.029...","[0.026106788, 0.04424294, 0.006456062, 0.00941...","[-0.018102376, -0.0074234153, 0.008787064, -0....","[0.015812408, 0.10102768, -0.009773148, -0.043...",0.176531,0.395928,0.051182
546,Take No

## Pridávanie vzťahov

3 vzťahy:

1 - konečný kontext vzoru X a začiatočný vzoru Y sú podobné1

2 - začiatočný kontext vzoru X a začiatočný vzoru Y sú podobné

3 - konečný kontext vzoru X a konečný vzoru Y sú podobné


Ak je podobnosť viac ako 0.6 tak vzťah predpokladám že existuje

In [136]:
df_final['result_start_1'] = 0
df_final['result_start_2'] = 0
df_final['start_start'] = 0
df_final['result_result'] = 0


df_final.loc[df_final['cos_similarity_result_start_1']>0.6, 'result_start_1'] = 1
df_final.loc[df_final['cos_similarity_result_start_2']>0.6, 'result_start_2'] = 1
df_final.loc[df_final['cos_similarity_start_start']>0.75, 'start_start'] = 1
df_final.loc[df_final['cos_similarity_result_result']>0.65, 'result_result'] = 1


## Vytváranie grafu pre 1. vzťah (konečný X a začiatočný Y)

In [137]:
values_1 = df_final[df_final['result_result']==1][['pattern_name1', 'pattern_name2']].values
pairs=[]
for value in values_1:
    pairs.append((value[0],value[1]))
pairs

[('Developer Controls Process', 'Feature Assignment'),
 ('Development Episode', 'Get On With It'),
 ('Development Episode', 'Informal Labor Plan'),
 ('Development Episode', 'LockEmUpTogether'),
 ('Get On With It', 'Divide And Conquer'),
 ('Diverse Groups', 'Self-Selecting Team'),
 ('EngageCustomers', 'Surrogate Customer'),
 ('Team Pride', 'Unity Of Purpose'),
 ('Distribute Work Evenly', 'Few Roles'),
 ('Divide And Conquer', 'Few Roles'),
 ('Divide And Conquer', 'Hallway Chatter'),
 ('Divide And Conquer', 'Upside-Down Matrix Management'),
 ('Few Roles', 'Producer Roles'),
 ('Move Responsibilities', 'Producer Roles'),
 ('Move Responsibilities', 'Responsibilities Engage'),
 ('Move Responsibilities', 'Three To Seven Helpers Per Role'),
 ('Producers In The Middle', 'Producer Roles'),
 ('Producer Roles', 'Responsibilities Engage'),
 ('Architect Also Implements', 'Architect Controls Product'),
 ('LockEmUpTogether', 'Stand-Up Meeting')]

In [97]:
values_1 = df_final[df_final['result_start_1']==1][['pattern_name1', 'pattern_name2']].values
values_2 = df_final[df_final['result_start_2']==1][['pattern_name1', 'pattern_name2']].values
pairs=[]
for value in values_1:
    pairs.append((value[0],value[1]))
for value in values_2:
    pairs.append((value[1],value[0]))
pairs

[('Community Of Trust', 'Developer Controls Process'),
 ('Completion Headroom', 'Work Split'),
 ('Day Care', 'Generics And Specifics'),
 ('Development Episode', 'Unity Of Purpose'),
 ('Development Episode', 'LockEmUpTogether'),
 ('Mercenary Analyst', 'Scenarios Define Problem'),
 ('Named Stable Bases', 'Private World'),
 ('Team Per Task', 'Holistic Diversity'),
 ('Team Per Task', 'Matron Role'),
 ('Work Flows Inward', 'Producers In The Middle'),
 ('Diverse Groups', 'Self-Selecting Team'),
 ('EngageCustomers', 'Scenarios Define Problem'),
 ('EngageCustomers', 'Surrogate Customer'),
 ('Engage Quality Assurance', 'Group Validation'),
 ("Conway's Law", 'LockEmUpTogether'),
 ('Coupling Decreases Latency', 'Distribute Work Evenly'),
 ('Coupling Decreases Latency', 'Few Roles'),
 ('Coupling Decreases Latency', 'Move Responsibilities'),
 ('Coupling Decreases Latency', 'Organization Follows Location'),
 ('Coupling Decreases Latency', 'Shaping Circulation Realms '),
 ('Coupling Decreases Latency

In [138]:
import networkx as nx
G = nx.DiGraph()
G.add_edges_from(pairs)

for node in G.nodes:
    language = df[df['pattern_names']==node]['pattern_language'].values
    if language == 'PiecemealGrowthPatternLanguage':
        G.nodes[node]['group']=0
    if language == 'ProjectManagmentPatternLanguage':
        G.nodes[node]['group']=1
    if language == 'PeopleAndCodePatternLanguage':
        G.nodes[node]['group']=2
    if language == 'OrganizationalStyleLanguage':
        G.nodes[node]['group']=3

In [139]:
import pyvis
from pyvis.network import Network

net = Network(notebook=True, directed=False)
net.from_nx(G)

net.show("patterns.html")

patterns.html


In [95]:
net.save_graph("patterns.html")

## ML (nepoužité)

Jeden zo spôsobov ako použiť získané data s ML je napríklad použitie regresie, v tomto prípade som skúsil logistickú regresiu kde ako vstup idú metriky podbnosti. Samozrejme model potrebuje aj label-y takže som skúsil použiť moje predtým vytvorené informácie, či sú vzoru v postupnosti alebo nie. Ale nakoľko som použil len jednu sekvenciu (aj to možno nie s úplne správnym uvažovaním) model nedokázal na testovacej vzroke správne určiť 3 dvojice vzorov, ktoré som identifikoval ako páry v postupnosti

In [411]:
X = df_final[['cos_similarity','euc_distance']]
y = df_final['sequence']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)


In [412]:
logreg = LogisticRegression(random_state=16)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [414]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[160,   0],
       [  3,   0]], dtype=int64)

## Čo ďalej

V prvom rade rožšíriť databázu, spravovať aj ostatné jazkyky vzorov, prípadne ich príbehy o postupnosti (sekvencii). Taktiež nájsť riešnie ako použiť modely neurónovej siete na klasifikáciu párov textov (kontextov) na základe ich podobnosti.

Nájdenie thresholdu pre vzťah na základe podbností vektorov.

Pri prípade používania alternatívneho riešenia (text ako vektor). Navrhnutie metódy, ktorá bude schopná klasifikovať vzťahy vzorov na zákade ich vektorovej podobnosti - napríklad pomocou unsupervised learning (clustering)

In [110]:
df_final[(df_final['cos_similarity_result_start_1']>0.65) | (df_final['cos_similarity_result_start_2']>0.65)].count()

pattern_name1                    25
pattern_name2                    25
resulting_embedding1             25
resulting_embedding2             25
starting_embedding1              25
starting_embedding2              25
cos_similarity_result_start_1    25
cos_similarity_start_start       25
cos_similarity_result_result     25
cos_similarity_result_start_2    25
result_start_1                   25
result_start_2                   25
start_start                      25
result_result                    25
dtype: int64

In [111]:
df_final[(df_final['cos_similarity_result_start_1']>0.5) | (df_final['cos_similarity_result_start_2']>0.5)].count()

pattern_name1                    389
pattern_name2                    389
resulting_embedding1             389
resulting_embedding2             389
starting_embedding1              389
starting_embedding2              389
cos_similarity_result_start_1    389
cos_similarity_start_start       389
cos_similarity_result_result     389
cos_similarity_result_start_2    389
result_start_1                   389
result_start_2                   389
start_start                      389
result_result                    389
dtype: int64

In [112]:
df_final[(df_final['cos_similarity_result_start_1']>0.6) | (df_final['cos_similarity_result_start_2']>0.6)].count()

pattern_name1                    66
pattern_name2                    66
resulting_embedding1             66
resulting_embedding2             66
starting_embedding1              66
starting_embedding2              66
cos_similarity_result_start_1    66
cos_similarity_start_start       66
cos_similarity_result_result     66
cos_similarity_result_start_2    66
result_start_1                   66
result_start_2                   66
start_start                      66
result_result                    66
dtype: int64

In [150]:
df_final[(df_final['cos_similarity_result_result']>0.5)].count()

pattern_name1                    296
pattern_name2                    296
resulting_embedding1             296
resulting_embedding2             296
starting_embedding1              296
starting_embedding2              296
cos_similarity_result_start_1    296
cos_similarity_start_start       296
cos_similarity_result_result     296
cos_similarity_result_start_2    296
result_start_1                   296
result_start_2                   296
start_start                      296
result_result                    296
dtype: int64